In [3]:
# %pip install --index-url https://download.pytorch.org/whl/cpu torch
# %pip install transformers
%pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 61.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 138.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import json
import time
import platform
import socket
import subprocess
from datetime import datetime

import requests
import torch
from transformers import AutoTokenizer, AutoModel

In [5]:

# ---------- Helpers ----------
def run(cmd):
    return subprocess.check_output(cmd, text=True).strip()

def get_imds_token(timeout=2):
    """Get IMDSv2 token (works when IMDS is enabled; required on many EC2 configs)."""
    try:
        r = requests.put(
            "http://169.254.169.254/latest/api/token",
            headers={"X-aws-ec2-metadata-token-ttl-seconds": "21600"},
            timeout=timeout,
        )
        if r.status_code == 200:
            return r.text
    except Exception:
        pass
    return None

def get_imds(path, token=None, timeout=2):
    """Query EC2 instance metadata. Returns None if not on EC2 / blocked."""
    url = f"http://169.254.169.254/latest/meta-data/{path}"
    headers = {}
    if token:
        headers["X-aws-ec2-metadata-token"] = token
    try:
        r = requests.get(url, headers=headers, timeout=timeout)
        if r.status_code == 200:
            return r.text
    except Exception:
        pass
    return None

In [6]:
token = get_imds_token()
instance_id = get_imds("instance-id", token)
instance_type = get_imds("instance-type", token)
az = get_imds("placement/availability-zone", token)

print("=== EC2 Proof ===")
print("hostname:", socket.gethostname())
print("local time:", datetime.now().isoformat(timespec="seconds"))
print("instance-id:", instance_id)
print("instance-type:", instance_type)
print("availability-zone:", az)

if instance_id is None:
    print("WARNING: Could not access EC2 metadata. You might not be on EC2, or IMDS is disabled/blocked.")
else:
    print("OK: EC2 metadata accessible (strong proof you're on an EC2 instance).")

# Optional extra evidence: show OS + kernel
print("\n=== System ===")
print("platform:", platform.platform())
try:
    print("uname -a:", run(["uname", "-a"]))
except Exception:
    pass

=== EC2 Proof ===
hostname: ip-10-192-1-105
local time: 2026-02-04T04:06:12
instance-id: i-0223bf6e9895a1cc4
instance-type: t3.medium
availability-zone: us-east-1c
OK: EC2 metadata accessible (strong proof you're on an EC2 instance).

=== System ===
platform: Linux-6.14.0-1018-aws-x86_64-with-glibc2.39
uname -a: Linux ip-10-192-1-105 6.14.0-1018-aws #18~24.04.1-Ubuntu SMP Mon Nov 24 19:46:27 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [7]:
print("\n=== Python & Packages ===")
print("python:", platform.python_version())
print("torch:", torch.__version__)
print("transformers:", __import__("transformers").__version__)
print("requests:", requests.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("cuda version:", torch.version.cuda)
    print("gpu name:", torch.cuda.get_device_name(0))


=== Python & Packages ===
python: 3.12.3
torch: 2.10.0+cpu
transformers: 5.0.0
requests: 2.32.5
cuda available: False


In [8]:
# ---------- BERT Smoke Test ----------
ckpt = "bert-base-uncased"
text = "Hello from EC2. This is a BERT smoke test."

tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModel.from_pretrained(ckpt)

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# CLS embedding (standard for sentence embedding demo)
cls = outputs.last_hidden_state[:, 0, :]
print("CLS embedding shape:", cls.shape)


Loading weights: 100%|█| 199/199 [00:00<00:00, 516.70it/s, Materializing param=pool
BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


CLS embedding shape: torch.Size([1, 768])


In [ ]:
out_dir = "outputs"
os.makedirs(out_dir, exist_ok=True)

payload = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "hostname": socket.gethostname(),
    "instance_id": instance_id,
    "instance_type": instance_type,
    "availability_zone": az,
    "python": platform.python_version(),
    "torch": torch.__version__,
    "transformers": __import__("transformers").__version__,
    "cuda_available": torch.cuda.is_available(),
    "checkpoint": ckpt,
    "text": text,
    "cls_embedding_shape": list(cls.shape),
    "cls_embedding_first5": cls[0, :5].tolist(),
}

path = os.path.join(out_dir, "ec2_bert_smoketest.json")
with open(path, "w", encoding="utf-8") as f:
    json.dump(payload, f, indent=2)

print("\nSaved proof file:", path)
print("Done ✅")